In [15]:
from mqtt import MQTT_connection
from IPython.display import display
import ipywidgets as widgets
import time, threading
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [18]:
class Controller:
    def __init__(self):
        self.mqtt = MQTT_connection()
        self.button = widgets.Button(description = "Switch Light Status")
        self.light_state = 0
        self.temperatures = []
        self.datetimes = []
        self.plot_data ={"time":range(len(self.temperatures)),"temperature":self.temperatures}        
        self.button.on_click(self.switch_lights)
        
        self.toggle = widgets.ToggleButton(description = "Switch Lights", value = bool(self.mqtt.light_state), 
                                            button_style = "info")
        self.toggle.observe(self.switch_lights, "value")
        display(self.toggle)
#         self.update_temperatures()
        self.update_thread = threading.Thread(target = self.update_temperatures, args = ())
        self.update_thread.start()
   
    def switch_lights(self, x):
        #x passed because of how the ipywidgets button passes stuff to the connected function
        self.mqtt.switch_lights()
               
    
    def update_temperatures(self):
        
        fig, ax = plt.subplots()
        ax.autoscale(enable = True, axis = "both", tight = True)
        ax.set_xlabel("Time")
        ax.set_ylabel("Temperature (C)")
        line, = ax.plot([],[])
        plt.show()
        self.alive = True
        counter = 0
        while self.alive:
            self.temperatures.append(self.mqtt.q.get())
            self.mqtt.q.task_done()
            self.datetimes.append(datetime.datetime.now())
            ax.set_title(f"The Lights are {self.mqtt.light_state}")
            
            counter +=1

            x_data = range(len(self.temperatures))
            line.set_xdata(self.datetimes)
            line.set_ydata(self.temperatures)
#                 self.ax.relim()
#                 self.ax.autoscale_view(tight = True)
            ax.set_xlim([self.datetimes[0],self.datetimes[-1]+datetime.timedelta(seconds = 5)])
            ax.set_ylim(ymin = np.min(self.temperatures) -1, ymax = np.max(self.temperatures)+1)
            fig.canvas.draw()
            time.sleep(0.5)
                                        
            
            
            
            

In [17]:
%matplotlib widget
c = Controller()

ToggleButton(value=False, button_style='info', description='Switch Lights')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …